In [ ]:
################################
########### Imports ############
################################

import sys
sys.path.append('../../python/')          # Defining path

import numpy as np                        
import matplotlib.pyplot as plt

import NGC5533_functions as nf            # Components
import 5005_widget               # Traced data of Galaxy NGC 5533
import fitting_NGC5533 as fitting         # Fitting parameters for best fit values

from ipywidgets import interactive, fixed, FloatSlider, HBox, Layout, Button, Label, Output, VBox
from IPython.display import display

import warnings
warnings.filterwarnings("ignore")         #ignore warnings



# Define components
def bulge(BX):
    return BX*rbulgev_fit

def disk(DX):
    return DX*rdiskv_fit

def halo(r_dat,rc,rho00):
    return nf.h_v(r_dat,rc,rho00,load=False)

def gas(GX):
    return GX*rgasv_fit

def totalcurve(rval,GX,BX,DX,rc,rho00):
    return np.sqrt((gas(GX)**2)
                   + (bulge(BX)**2) 
                   + (disk(DX)**2)
                   + (halo(rval,rc,rho00)**2))


# In[9]:


#best fitted prefactor values for each component, to be used as default (initial) values for widget sliders
g_dict = g_fit.best_values
best_GX = g_dict['GX']
best_BX = g_dict['BX']
best_DX = g_dict['DX']
best_rc = g_dict['rc']
best_rho00 = g_dict['rho00']


# In[10]:


# Define plotting function
def f(GX,BX,DX,rc,rho00):
    
    # Define r
    xmax=12 #kpc
    r = np.linspace(0,11.2,19)
    
    # Plot
    plt.figure(figsize=(11,6))
    plt.xlim(0,xmax)
    plt.ylim(0,360)
    
    plt.errorbar(r_dat,v_dat,yerr=v_err1,fmt='bo',label='Data')
    plt.plot(rbulger,bulge(BX),label=("Bulge"),color='orange')
    plt.plot(rdiskr,disk(DX),label=("Disk"),color='purple')
    plt.plot(rval,halo(rval,rc,rho00),label=("Halo"),color='green')
    plt.plot(rgasr,gas(GX),label=("Gas"),color='blue')
    plt.plot(r_dat,totalcurve(rval,GX,BX,DX,rc,rho00),label=("Total Curve"),color='red')
    plt.plot(r_dat,bestg,'k',label='proper Total Fit')
    plt.title("Interactive Rotation Curve - Galaxy: NGC 5005")
    
    #plt.plot(rval,halo_curve,'r-',label='Correct Halo')
    #plt.plot(rbulger,g_b*rbulgev,label='Correct bulge')
    plt.xlabel("Radius (kpc)")
    plt.ylabel("Velocity (km/s)")
    
    
    plt.legend(loc='lower right')
    
     
    #chi squared button commented out for now bc tiny graph
    '''
    # Chi squared and reduced chi squared
    # Residuals
    
    r = np.linspace(0,11.2,69)
    residuals = v_dat - totalcurve(r_dat,GX,BX,DX,rc,rho00)
    # Determining errors
    errors = v_err1**2 #second term is inclination uncertainty
    # Chi squared
    chisquared = np.sum(residuals**2/errors**2)
    #chisquared = stats.chisquare(v_dat,totalcurve(r,M,bpref,dpref,rc,rho00,gpref))
    reducedchisquared = chisquared * (1/(len(r_dat)-6))
    
   
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    plt.text(80,170,r"$\chi^2$: {:.5f}".format(chisquared)+'\n'+r"Reduced: {:.5f}".format(reducedchisquared),bbox=props)
    #plt.text(80,150,,bbox=props)
   
    plt.annotate('Can you get the Reduced $\chi^2$ to zero?',
            xy=(0, 0), xytext=(510, 420),
            xycoords=('axes fraction', 'figure fraction'),
            textcoords='offset points',
            size=22, ha='left')

    plt.text(105,300,"In a strictly statistical sense, a Reduced $\chi^2$ of zero", fontsize=13),
    plt.text(105,275,"represents a perfect fit; however, a Reduced $\chi^2$ much", fontsize=13),
    plt.text(105,250,"less than one often indicates that more parameters were", fontsize=13),
    plt.text(105,225,"used than represent the physical system. In this fit,", fontsize=13),
    plt.text(105,200,"the Reduced $\chi^2$ is around 1 due primarily to the", fontsize=13),
    plt.text(105,175,"gas component, which is actually fixed (according", fontsize=13),
    plt.text(105,150,"to the physical characteristics of the Hydrogen gas", fontsize=13),
    plt.text(105,125,"it represents; see our video for more details). Here,", fontsize=13),
    plt.text(105,100,"we allow you to break the laws of nature and scale", fontsize=13),
    plt.text(105,75,"each component as you please. See how 'good' of", fontsize=13),
    plt.text(105,50,"a fit you can get!", fontsize=13)
    plt.annotate('Data source: [PUT SOURCE HERE]',
            xy=(0, 0), xytext=(0,5),
            xycoords=('axes fraction', 'figure fraction'),
            textcoords='offset points',
            size=10, ha='left', va='bottom')
    '''
    plt.show()


# In[11]:


# Appearance
style = {'description_width': 'initial'}
layout = {'width':'600px'}

# Define slides

GX = FloatSlider(min=0, max=5, step=0.1, 
                    value=best_GX,
                    description='Gas Prefactor', 
                    readout_format='.2f', 
                    orientation='horizontal', 
                    style=style, layout=layout)

BX = FloatSlider(min=0, max=5, step=0.1, 
                    value=best_BX, 
                    description='Bulge Prefactor', 
                    readout_format='.2f', 
                    orientation='horizontal', 
                    style=style, layout=layout)

DX = FloatSlider(min=0, max=5, step=0.1, 
                    value=best_DX, 
                    description='Disk Prefactor', 
                    readout_format='.2f', 
                    orientation='horizontal', 
                    style=style, layout=layout)

#rc = FloatSlider(min=0, max=5, step=0.1, value=best_rc, description='Halo Core Radius [kpc]', readout_format='.2f', orientation='horizontal', style=style, layout=layout)
rc = fixed(best_rc)
#rho00 = fixed(best_rho00)

rho00 = FloatSlider(min=0, max=1e9, step=1e7, 
                    value=best_rho00, 
                    description=r'Halo Surface Density [$M_{\odot} / pc^3$]', 
                    readout_format='.2e', 
                    orientation='horizontal', 
                    style=style, layout=layout)

# Interactive widget
def interactive_plot(f):
    interact = interactive(f,
                               GX = GX,
                               BX = BX, 
                               DX = DX, 
                               rc = rc,
                               rho00 = rho00,
        
                               continuous_update=False)
    return interact

# Button to revert back to Best Fit
button = Button(
    description="Best Fit",
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    icon='check')
out = Output()

def on_button_clicked(_):
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    BX.value = best_BX
    DX.value = best_DX
    rho00.value = best_rho00
    GX.value = best_GX

button.on_click(on_button_clicked)

# displaying button and its output together
VBox([button,out,interactive_plot(f)])


# In[ ]:




